# Template Jupyter Notebook

In [28]:
import json
import pandas as pd
from pathlib import Path
import os

In [29]:
def parse_json(path: str):
    with open(path, "r", encoding="utf-8") as file:
        raw_data = json.load(file)
    
    messages = pd.DataFrame(raw_data.get("messages",[]))
    participants = pd.DataFrame(raw_data["participants"])
    thread_path = raw_data.get("thread_path",os.path.basename(path))
    return messages, participants, thread_path

In [30]:
def get_conversation(messages: pd.DataFrame, totalConversation: dict):
    for index, row in messages.iterrows():
        sender = row.get("sender_name")
        content = row.get("content")
        if(content == None):
            continue
    totalConversation.setdefault(sender, []).append(content)
    return totalConversation

In [31]:
def update_conversation_counts(messages: pd.DataFrame, participants: pd.DataFrame, totalConversation: dict):
    if len(participants) != 2:
        return totalConversation

    p1 = participants["name"].iloc[0]

    if p1 not in totalConversation:
        totalConversation[p1] = {"sent": 0, "received": 0}

    for _, row in messages.iterrows():
        sender = row.get("sender_name")
        content = row.get("content")
        if content is None:
            continue    
    if sender == p1:
        totalConversation[p1]["sent"] += 1
    else:
        totalConversation[p1]["received"] += 1

    return totalConversation

In [ ]:
totalConversation={}
# folder_path = r"c:\Users\nlhng\OneDrive\Desktop\UVEC2025\dev\test"
for file_path in Path(folder_path).glob("*.json"):
    messages, participants, thread_path = parse_json(file_path)

    if len(participants) == 2:
        totalConversation = update_conversation_counts(messages, participants, totalConversation)

    df = pd.DataFrame.from_dict(totalConversation, orient="index")
    df.index.name = "participant"
    print(df)
    ## divide recieved by sent then sort

              sent  received
participant                 
Omar Madhani     0         1
              sent  received
participant                 
Omar Madhani     0         1
Kaitlynð      1         0
              sent  received
participant                 
Omar Madhani     0         1
Kaitlynð      1         0
Rowan            0         1
